

# Wrangle and Analyze Data

_We Rate Dogs dataset Wrangling code...By Randy Jackson_

## Introduction

WeRateDogs is a Twitter account that rates people's dogs with a humorous comment about the dog. These ratings almost always have a denominator of 10. The numerators, though? Almost always greater than 10. 11/10, 12/10, 13/10, etc. Why? Because "they're good dogs Brent." WeRateDogs has over 4 million followers and has received international media coverage.  

The datasets we wrangling are a tweet archive of Twitter user @dog_rates, also known as WeRateDogs. This data is supplemented with additional data via the Twitter API and and image predictions file which is the output of running the images in the WeRateDogs Twitter archive through a neural network that can classify breeds of dogs.

This Juyter notebook provides the code for gathering, assessing, cleaning, analyzing, and visualizing data for the 'We Rate Dogs' Wrangle and Analyze data project.  Two other supporting notebooks are included:

- wrangle_report.ipynb: Provides documentation for data wrangling steps: gather, assess, and clean.
- act_report.ipnb: Included the subsequent Insights and Visualizations.

## Gather Data

In [1]:
import numpy as np
import pandas as pd
import json
import tweepy
import requests
from scipy import stats
import io
import csv # read and write csv files
from datetime import datetime # operations to parse dates
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import re


### Twitter Archive

In [2]:
# Contains tweets with extra info already parsed from text
df_archive = pd.read_csv('twitter-archive-enhanced.csv')
df_archive.shape

(2356, 17)

### Image Predictions (URL)

In [3]:
# Predictions of dog breeds
url_image_predictions = ('https://d17h27t6h515a5.cloudfront.net/topher/2017/August/'
                         '599fd2ad_image-predictions/image-predictions.tsv')

# Use requests library to get data from URL
# Create dataframe from tabbed-seperated_values (TSV) files
with requests.Session() as s:
    download = s.get(url_image_predictions) #Use get() to download the data
    decoded_content = download.content.decode('utf-8') #make sure it's properly encoded
    df_images = pd.read_csv(io.StringIO(decoded_content) , delimiter='\t') #convert to dataframe
 
df_images.shape

(2075, 12)

### Tweets (Twitter API)

In [4]:
consumer_key = 'YOUR CONSUMER KEY'
consumer_secret = 'YOUR CONSUMER SECRET'
access_token = 'YOUR ACCESS TOKEN'
access_secret = 'YOUR ACCESS SECRET'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

#api = tweepy.API(auth)
api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)


In [146]:
# Store data on tweet_json.txt

with open('tweet_json.txt', 'w') as outfile: 
    for tweet_id in df_archive['tweet_id']:
        try:
            tweet = api.get_status(tweet_id)
            # Convert to JSON and write
            json.dump(tweet._json, outfile)
            outfile.write("\n")       
        except tweepy.TweepError as te:
            if te.api_code==88: #rate limit exceeded error code
                time.sleep(60 * 15)
                continue
            else:
                print("Failed to get Tweet ID:{0}, Reason:{1}".format(tweet_id, te.reason))
                continue
    

Failed to get Tweet ID:888202515573088257, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:873697596434513921, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:869988702071779329, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:866816280283807744, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:861769973181624320, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:842892208864923648, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:837012587749474308, Reason:[{'code': 34, 'message': 'Sorry, that page does not exist.'}]
Failed to get Tweet ID:827228250799742977, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed to get Tweet ID:802247111496568832, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]
Failed t

**Saved Output from Tweepy API code**

Failed to get Tweet ID:888202515573088257, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:873697596434513921, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:869988702071779329, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:866816280283807744, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:861769973181624320, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:842892208864923648, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:837012587749474308, Reason:[{'code': 34, 'message': 'Sorry, that page does not exist.'}]  
Failed to get Tweet ID:827228250799742977, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:802247111496568832, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Failed to get Tweet ID:775096608509886464, Reason:[{'code': 144, 'message': 'No status found with that ID.'}]  
Rate limit reached. Sleeping for: 653  
Rate limit reached. Sleeping for: 678  

In [5]:
# Tweet atrributes to save in dataframe
tweet_attr_list = ['id', 'favorite_count', 'retweet_count']
data = []

with open('tweet_json.txt', 'r') as readfile: 
    # Read in JSON line and convert to dict
    tweet_json = readfile.readline()
    
    # Read line by line into DataFrame
    while tweet_json:
        tweet_dict = json.loads(tweet_json)
        data_row = collections.OrderedDict((k, tweet_dict[k]) for k in tweet_attr_list)
        data.append(data_row)
        
        # Read in JSON line and convert to dict
        tweet_json = readfile.readline() 
            
df_tweets = pd.DataFrame.from_dict(data)
df_tweets.shape

(2346, 3)

## Assess

I started first by importing the data into Microsoft Access so that I could better scroll and visually inspect the data.  I will note those findings at the end of the asseement section when I note the observations.

Here we will start by using the Panda functions to look at the data programmatically.  Let's start by looking at table's info() output, sampling of the data and data types, we'll use the twitter developers documentation to help determine the proper data type _https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object_ 


In [6]:
df_archive.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None,None,None,None,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax,None,None,None,None
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None,None,None,None,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey,None,None,None,None
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie,doggo,None,None,None


In [7]:
df_archive.shape

(2356, 17)

In [8]:
df_archive.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

In [9]:
df_archive.rating_numerator.unique()

array([  13,   12,   14,    5,   17,   11,   10,  420,  666,    6,   15,
        182,  960,    0,   75,    7,   84,    9,   24,    8,    1,   27,
          3,    4,  165, 1776,  204,   50,   99,   80,   45,   60,   44,
        143,  121,   20,   26,    2,  144,   88], dtype=int64)

In [10]:
df_archive.rating_denominator.unique()

array([ 10,   0,  15,  70,   7,  11, 150, 170,  20,  50,  90,  80,  40,
       130, 110,  16, 120,   2], dtype=int64)

In [11]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML

df_archive.name.sample(50)


843           None
848           Juno
508           None
919           None
1152        Lillie
125           None
550         Django
1319          None
1526         Phred
1024        Shiloh
1993         Caryl
695          Logan
2315    Christoper
1493          None
1229         Piper
154           None
948           None
928          Keith
2256        Calvin
2157          None
483         Cooper
1077          None
610        Pancake
1171          None
1870          None
2031          None
1636          None
1147        Sprout
1180          Lucy
1809           Ash
572           None
37            None
1621       Bruiser
1364          Luna
1520      Vinscent
23          Canela
1914          None
1025            an
2080     Remington
1156          None
2045           Jax
281           None
2006          None
1882          None
88            None
756        Strider
1860          None
134           Lili
356          Lilly
1329           Max
Name: name, dtype: object

<div class="alert alert-block alert-info">
It appears the invalid names are all lower case
</div>


In [12]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML

df_archive.in_reply_to_status_id.sample(50)

2297             NaN
155              NaN
463              NaN
0                NaN
2083             NaN
2080             NaN
2081             NaN
1158             NaN
1440             NaN
872              NaN
121              NaN
975              NaN
1942             NaN
2330             NaN
2315             NaN
1839             NaN
2092             NaN
1456             NaN
2065             NaN
2249             NaN
1810             NaN
169              NaN
2261             NaN
2289             NaN
968              NaN
2                NaN
55      8.816070e+17
1678             NaN
486              NaN
2020             NaN
2351             NaN
2182             NaN
850              NaN
116              NaN
474              NaN
2235             NaN
1060             NaN
1351             NaN
1428             NaN
682              NaN
1964             NaN
1569             NaN
2158             NaN
1010             NaN
686              NaN
755              NaN
1307             NaN
1684         

In [13]:
dog_stage_cnts =(df_archive.query("doggo !='None'").shape[0] + df_archive.query("floofer !='None'").shape[0]
    + df_archive.query("pupper !='None'").shape[0] + df_archive.query("puppo !='None'").shape[0])
dog_stage_cnts
                                                   

394

<div class="alert alert-block alert-info">
Most of the dogs have no stage entered
</div>

In [14]:
df_images.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True


In [15]:
df_images.shape

(2075, 12)

In [16]:
df_images.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [17]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML
df_images.p1.sample(50)

1850    Chesapeake_Bay_retriever
1563          miniature_pinscher
1074              Siberian_husky
1756                  Eskimo_dog
1788              Border_terrier
1902                  pencil_box
595                    Chihuahua
1858                    ice_bear
1322                      geyser
40               German_shepherd
345                          pug
916                Border_collie
882                      Samoyed
1591                        chow
1073                    malinois
1196            golden_retriever
627                     Pembroke
198                    seat_belt
1503                        chow
478                     Pembroke
1361                      cougar
1230                   seat_belt
563           miniature_pinscher
1179                         pug
415                     Pembroke
594                    porcupine
1884                    Cardigan
1779                       teddy
1995              cocker_spaniel
1042                    ice_bear
454       

<div class="alert alert-block alert-info">
Like the invalid dog names, the predicted animal/breed for invalid breeds appears to be lower case
</div>

<div class="alert alert-block alert-info">
There are fewer rows here than in the twitter archive file 2075 versus 2356
</div>

In [18]:
df_tweets.head(10)

,id,favorite_count,retweet_count
0,892420643555336193,39060,8673
1,892177421306343426,33457,6372
2,891815181378084864,25219,4232
3,891689557279858688,42426,8776
4,891327558926688256,40582,9553
5,891087950875897856,20359,3170
6,890971913173991426,11936,2113
7,890729181411237888,66013,19217
8,890609185150312448,27954,4330
9,890240255349198849,32166,7547


In [19]:
df_tweets.shape

(2346, 3)

In [20]:
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2346 entries, 0 to 2345
Data columns (total 3 columns):
id                2346 non-null int64
favorite_count    2346 non-null int64
retweet_count     2346 non-null int64
dtypes: int64(3)
memory usage: 55.1 KB


In [21]:
df_images[df_images.duplicated('tweet_id')]

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog


In [22]:
df_images[df_images.duplicated('jpg_url')].head(6)


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
1297,752309394570878976,https://pbs.twimg.com/ext_tw_video_thumb/67535...,1,upright,0.303415,False,golden_retriever,0.181351,True,Brittany_spaniel,0.162084,True
1315,754874841593970688,https://pbs.twimg.com/media/CWza7kpWcAAdYLc.jpg,1,pug,0.272205,True,bull_mastiff,0.251530,True,bath_towel,0.116806,False
1333,757729163776290825,https://pbs.twimg.com/media/CWyD2HGUYAQ1Xa7.jpg,2,cash_machine,0.802333,False,schipperke,0.045519,True,German_shepherd,0.023353,True
1345,759159934323924993,https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg,1,Irish_terrier,0.254856,True,briard,0.227716,True,soft-coated_wheaten_terrier,0.223263,True
1349,759566828574212096,https://pbs.twimg.com/media/CkNjahBXAAQ2kWo.jpg,1,Labrador_retriever,0.967397,True,golden_retriever,0.016641,True,ice_bear,0.014858,False
1364,761371037149827077,https://pbs.twimg.com/tweet_video_thumb/CeBym7...,1,brown_bear,0.713293,False,Indian_elephant,0.172844,False,water_buffalo,0.038902,False


<div class="alert alert-block alert-info">
No tweet ids are duplicated but some of the images are, 66 images a referenced twice in the image file
</div>

So here are my observances, some were noted during the visual assessment I did offline using Mircosoft Access and others were noted during the programmatic assessment.  These observances will be address in the order stated, quality issues completness, followed by tidiness, then the remaining quality issues.

#### Completeness

Quality - Missing 281 records from Predictive Image file (2356 in Twitter Archive minus 2075 in the Prwedictive Image file).   
Quality - Missing 10 records from Tweets data due to errors.  

#### Tidiness

Tidiness - Dog stages are stored in four columns (doggo, floofer, pupper, and puppo).  
Tidiness - One archive record may contain information for more than a single dog (two dogs, two ratings, two stages)  
Tidiness - Mulitple prediction columns and the inclusion non-dog information may have limited value.  
Tidiness - Multiple types of observational units are stored in the same table, Twitter data and Dog data are comingled

#### Quality

Quality - Data contains retweets  
Quality - Data contains values unrelated to tweets or dogs   
Quality - Some dogs hove no pictures  
Quality - Some dogs have bad dog names   
Quality - `Source` column has no value, and multiple values in one column   
Quality - `Expanded URLs` has no value, and multiple values in one column        
Quality - `Image num` has no value    
Quality - `timestamp` has the wrong data type      
Quality - Data types for `in_reply_to_status_id` and `in_reply_to_user_id` could be changed to match Twitter API   
Quality - `in_reply_to_status_id` and `in_reply_to_user_id` are sparsely populated  
Quality - Dog breed names are seperated by underscore and have inconsistent case
    

## Clean

<div class="alert alert-block alert-info">
Start by making copies of our data
</div>

In [23]:
df_archive_copy = df_archive.copy
df_images_copy = df_images.copy
df_tweets_copy = df_tweets.copy

<div class="alert alert-block alert-info">
Quality - Missing 281 records from Predictive Image file (2356 in Twitter Archive minus 2075 in the Predictive Image file)
</div>

Unfortunately, there's isnt anything we can do about these rows yet.  We don't have access to the prediction model to try to obtain predictions for the remaining rows.  It appears that roughly 100 of these rows are associated with retweets and will later be removed which will reduce the discrepancy.  It is also possible that a number of then are aligned with images that are not dogs which I also plan to remove.  We will revisit this later. 

<div class="alert alert-block alert-info">
Quality - Missing 10 records from Tweets data due to errors.

NOTE:  This cleaning step turned out to be uncessary.  Later analysis determined that the 10 records in error were either related dogs with no images or re-tweets, each of which was subsequently removed.  However, I left the code in to illustrate and example of the process of imputing missing data.
</div>

#### Define

Tweets corresponding to a few tweet IDs in the archive may have been deleted and led to errors. In my case there were 10, we will add these 10 records to the tweets file and impute their favorite count and retween count using the mean

#### Code

In [24]:
df_tweets.head(10)

,id,favorite_count,retweet_count
0,892420643555336193,39060,8673
1,892177421306343426,33457,6372
2,891815181378084864,25219,4232
3,891689557279858688,42426,8776
4,891327558926688256,40582,9553
5,891087950875897856,20359,3170
6,890971913173991426,11936,2113
7,890729181411237888,66013,19217
8,890609185150312448,27954,4330
9,890240255349198849,32166,7547


In [25]:
df_m = pd.merge(df_archive, df_tweets, how = 'left', left_on='tweet_id', right_on='id' )

In [26]:
df_m =df_m[df_m['favorite_count'].isnull()]
mlist=df_m['tweet_id'].tolist()
mlist

[888202515573088257,
 873697596434513921,
 869988702071779329,
 866816280283807744,
 861769973181624320,
 842892208864923648,
 837012587749474308,
 827228250799742977,
 802247111496568832,
 775096608509886464]

In [27]:
favorite_count_mean=df_tweets['favorite_count'].mean()
retweet_count_mean=df_tweets['retweet_count'].mean()
print(favorite_count_mean)
print(retweet_count_mean)
a=np.rint(np.repeat(favorite_count_mean, 10))
b=np.rint(np.repeat(retweet_count_mean, 10))
a
b

8109.689258312021
3057.793691389599


array([ 3058.,  3058.,  3058.,  3058.,  3058.,  3058.,  3058.,  3058.,
        3058.,  3058.])

In [28]:
df_tmp=pd.DataFrame()
df_tmp['id']=mlist
df_tmp['favorite_count']=a
df_tmp['retweet_count']=b
df_tmp

,id,favorite_count,retweet_count
0,888202515573088257,8110.0,3058.0
1,873697596434513921,8110.0,3058.0
2,869988702071779329,8110.0,3058.0
3,866816280283807744,8110.0,3058.0
4,861769973181624320,8110.0,3058.0
5,842892208864923648,8110.0,3058.0
6,837012587749474308,8110.0,3058.0
7,827228250799742977,8110.0,3058.0
8,802247111496568832,8110.0,3058.0
9,775096608509886464,8110.0,3058.0


In [29]:
df_tweets.shape[0]
df_tweets = pd.concat([df_tweets, df_tmp])


#### Test

In [30]:
df_m = pd.merge(df_archive, df_tweets, how = 'left', left_on='tweet_id', right_on='id' )
df_m =df_m[df_m['favorite_count'].isnull()]
mlist=df_m['tweet_id'].tolist()
mlist
df_tweets.shape[0]

2356

Rerunning the outer join we get no unmapped IDs, and the number of rows is now equal

<div class="alert alert-block alert-info">
Tidiness - Dog rating are stored in four columns (doggo, floofer, pupper, and puppo)   

Tidiness - One archive record may contain information for more than a single dog (two dogs, two ratings, two stages)  
(this is partially addressed by my approach as each stage will exist in its own rows) 
</div>

#### Define

Use the melt method to consolidate dog stages into a single feature and delete the old columns.

#### Code

In [31]:
df_archive.shape[0]

2356

In [32]:
df_stage_slice = df_archive.loc[:, ['tweet_id', 'doggo', 'floofer', 'pupper', 'puppo']]

In [33]:
df_stage=pd.melt(df_stage_slice,id_vars='tweet_id', value_name='dog_stage') #melt the data, from wide to long
df_stage=df_stage.drop(['variable'], axis=1) #drop the variable column created in the melt step

df_stage.drop(df_stage[df_stage.dog_stage =='None'].index, inplace=True)


#### Test

In [34]:
df_stage.shape[0]


394

In [35]:
dog_stage_cnts =(df_archive.query("doggo !='None'").shape[0] + df_archive.query("floofer !='None'").shape[0]
    + df_archive.query("pupper !='None'").shape[0] + df_archive.query("puppo !='None'").shape[0])
dog_stage_cnts

394

In [36]:
df_archive = df_archive.drop(['doggo','floofer','pupper','puppo'], 1)

In [37]:
df_archive.columns.unique

<bound method Index.unique of Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'name'],
      dtype='object')>

This number is equal to the total count of dog stages in the orginal data accross all 4 columns.  Now the creation of a single column was correctly created earlier but it's currently in its own dataframe,  As part of our tidiness work we will move this data into another table dedicated to dog observations and delete the 4 columns from the archive table which will be dedicated to twitter observations.  This will also help account for the 12 records in the archive table that now have data in two stages. Our final stucture will provide one record per dog observation.

<div class="alert alert-block alert-info">
Tidiness- Mulitple prediction columns and the inclusion non-dog information may have limited value. - Step 1, Collapse the prediction columns
</div>

#### Define

Create and excute a function to apply to the dataframe tha will inspect each probablity related column and store the first true value and the associated probabilty, this info will be used to append new columns and the old ones will be dropped

#### Code

In [38]:
print(df_images.query("p1_conf<p2_conf").shape[0]) 
print(df_images.query("p2_conf<p3_conf").shape[0])
print(df_images.query("p1_dog==False & p2_dog==True").shape[0]) 
print(df_images.query("p1_dog==False & p2_dog==False & p3_dog==True").shape[0]) 

#this confirms the probablities are sequenced, with p1 always
#greater than p2, and that the true value has the greatest c.i.

0
0
155
64


In [39]:
# We will store the first true algorithm with it's level of confidence
predicted_breed = []
confidence_interval = []    

# Get_prediction_confidence function:
# search the first true algorithm and append it to a list with it's level of confidence
# if false prediction_algorthm will have a value of NaN
def get_prediction_confidence(dataframe):
    
    if dataframe['p1_dog'] == True:
        predicted_breed.append(dataframe['p1'])
        confidence_interval.append(dataframe['p1_conf'])
    elif dataframe['p2_dog'] == True:
        predicted_breed.append(dataframe['p2'])
        confidence_interval.append(dataframe['p2_conf'])
    elif dataframe['p3_dog'] == True:
        predicted_breed.append(dataframe['p3'])
        confidence_interval.append(dataframe['p3_conf'])
    else:
        predicted_breed.append('None')
        confidence_interval.append(0)
    
    return

In [40]:
df_images.apply(get_prediction_confidence, axis=1);
df_images['predicted_breed'] = predicted_breed;
df_images['confidence_interval'] = confidence_interval;


In [41]:
df_images.head(10)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,predicted_breed,confidence_interval
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True,miniature_pinscher,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137,True,English_springer,0.263788,True,Greater_Swiss_Mountain_dog,0.016199,True,Bernese_mountain_dog,0.651137
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,box_turtle,0.933012,False,mud_turtle,0.045885,False,terrapin,0.017885,False,None,0.000000
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517,True,Tibetan_mastiff,0.058279,True,fur_coat,0.054449,False,chow,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,shopping_cart,0.962465,False,shopping_basket,0.014594,False,golden_retriever,0.007959,True,golden_retriever,0.007959
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493,True,komondor,0.192305,True,soft-coated_wheaten_terrier,0.082086,True,miniature_poodle,0.201493


In [42]:
df_images = df_images.drop(['p1','p1_conf','p1_dog','p2','p2_conf','p2_dog','p3','p3_conf','p3_dog'], 1)

In [43]:
df_images.head(10)

,tweet_id,jpg_url,img_num,predicted_breed,confidence_interval
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311
5,666050758794694657,https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg,1,Bernese_mountain_dog,0.651137
6,666051853826850816,https://pbs.twimg.com/media/CT5KoJ1WoAAJash.jpg,1,None,0.000000
7,666055525042405380,https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg,1,chow,0.692517
8,666057090499244032,https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg,1,golden_retriever,0.007959
9,666058600524156928,https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg,1,miniature_poodle,0.201493


#### Test

In [44]:
len(df_images.query("predicted_breed !='None' & confidence_interval==0"))
len(df_images.query("predicted_breed =='None' & confidence_interval>0"))

0

We have now collapsed the columns, so all our non-empty values have probabilies greater than 0 and none of our 0 proablities rows are empty.  This means that all of our non-dog images are 'None' **AND** 0, we will later remove them leaving **ONLY DOGS WITH IMAGES**

<div class="alert alert-block alert-info">
Tidiness- Mulitple prediction columns and the inclusion non-dog information may have limited value. Step 2, Remove the non-dog images
</div>

#### Define

Now that we have now collapsed the columns, all of our non-dog images are 'None' **AND** 0, in this step we remove them leaving **ONLY DOGS WITH IMAGES**

#### Code

In [45]:
len(df_images.query("predicted_breed =='None'"))


324

In [46]:
df_images.drop(df_images[df_images.predicted_breed =='None'].index, inplace=True)

In [47]:
df_images
df_images.shape

(1751, 5)

#### Test

In [48]:
len(df_images.query("predicted_breed =='None' or confidence_interval==0"))

0

This means that all of our non-dog images, records with either 'None' or  0 confidence interval no longer exist in the data, and we are left with only pictures of dogs along with their highest confidence interval, and we have **1751** rows (2075 minus the 324 non-dog rows)

<div class="alert alert-block alert-info">
Quality - Some dogs have bad dog names. Step 1, fix names
</div>

#### Define

Some of the dog names appear to be bad, they all appear to be lower case words.  The dog names in the text are usually accompanied by several intoductory phrases.  We will loop through the names looking for these phrases, assume the next word is the dogs name and replace the name. We will limit the change only to lower case names in order to reduce the potential of altering good names.

#### Code

In [49]:
df_archive.head(10)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin
5,891087950875897856,NaN,NaN,2017-07-29 00:08:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Here we have a majestic great white breaching ...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891087950...,13,10,None
6,890971913173991426,NaN,NaN,2017-07-28 16:27:12 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Jax. He enjoys ice cream so much he gets ...,NaN,NaN,NaN,"https://gofundme.com/ydvmve-surgery-for-jax,ht...",13,10,Jax
7,890729181411237888,NaN,NaN,2017-07-28 00:22:40 +0000,"<a href=""http://twitter.com/download/iphone"" r...",When you watch your owner call another dog a g...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890729181...,13,10,None
8,890609185150312448,NaN,NaN,2017-07-27 16:25:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Zoey. She doesn't want to be one of th...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890609185...,13,10,Zoey
9,890240255349198849,NaN,NaN,2017-07-26 15:59:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Cassie. She is a college pup. Studying...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/890240255...,14,10,Cassie


In [50]:
df_archive[df_archive['name'].str.islower() ==True].shape[0]

109

In [51]:
# Inspect the text for one of the following

#This is [name] ..
#Meet [name] ..
#Say hallo to [name] ..
#Here we have [name] ..
#.. named [name] ..

dog_names = []

def get_dog_name(dataframe):
    txt=dataframe['text']
    cname=dataframe['name']
    # Start with 'This is ' and the fisrt letter of the name is uppercase
    if txt.startswith('This is ') and re.match(r'[A-Z].*', txt.split()[2]):
        if cname.islower():
            dog_names.append(txt.split()[2].strip(',').strip('.'))
        else:
            dog_names.append(cname) #preserve existing name
    # Start with 'Meet ' and the fisrt letter of the name is uppercase
    elif txt.startswith('Meet ') and re.match(r'[A-Z].*', txt.split()[1]):
        if cname.islower():
            dog_names.append(txt.split()[1].strip(',').strip('.'))
        else:
            dog_names.append(cname) #preserve existing name        
    # Start with 'Say hello to ' and the fisrt letter of the name is uppercase
    elif txt.startswith('Say hello to ') and re.match(r'[A-Z].*', txt.split()[3]):
        if cname.islower():
            dog_names.append(txt.split()[3].strip(',').strip('.'))
        else:
            dog_names.append(cname) #preserve existing name          
    # Start with 'Here we have ' and the fisrt letter of the name is uppercase
    elif txt.startswith('Here we have ') and re.match(r'[A-Z].*', txt.split()[3]):
        if cname.islower():
            dog_names.append(txt.split()[3].strip(',').strip('.'))
        else:
            dog_names.append(cname) #preserve existing name         
    # Contain 'named' and the fisrt letter of the name is uppercase
    elif 'named' in txt and re.match(r'[A-Z].*', txt.split()[txt.split().index('named') + 1]):
        if cname.islower():
            dog_names.append(txt.split()[txt.split().index('named') + 1].strip(',').strip('.'))
        else:
            dog_names.append(cname) #preserve existing name        
    # No name specified or other style
    else:
        dog_names.append('None')
        

# Save the result in a new column 'dog_name'

df_archive.apply(get_dog_name, axis=1)
df_archive['dog_name'] = dog_names




#### Test

In [52]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML
df_archive.name.sample(100)

1682     Charlie
2248      Shaggy
1382           a
571      Wallace
710        Rusty
944         None
564      Severus
2054        None
2113        Hank
2129      Sandra
2162      Ronduh
1465     Oliviér
1949        None
115         Zoey
748         None
807         None
701         None
1115     Aldrick
2178        None
1990    Leonidas
120      Stanley
1125     Charles
1638        None
109         Dawn
566         None
751         None
786       Finley
864        Buddy
1605        None
1315       Katie
          ...   
1601     Hammond
443         Anna
1984     Jimison
1085        None
260      Stephan
916       Oliver
376       Sailer
309         Lola
31       Waffles
2033        None
1004           a
2288        None
1253        Sora
2053    Penelope
606        Davey
1361           a
890         None
1381        Chet
1910       Jonah
2216       Spark
1961       Steve
580         None
2311           a
1023      Shaggy
2148    Clarence
1654        None
1879         Moe
378         Ko

In [53]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML
df_archive.dog_name.sample(100)

2046    Alejandro
1485       Dunkin
474         Comet
660         Mabel
122         Gizmo
1837         None
1884      Tedders
1090         None
486           Alf
2165         Chip
1856        Sammy
1164       Jangle
913          None
2093      Traviss
1739      William
448         Sunny
277          None
490          Gary
2089         None
1519      Peaches
310          None
976           Jax
2335         None
1987        Oscar
641          Juno
779          Blue
2059       Sparky
2056         None
2284         None
1794        Derek
          ...    
1415        Rusty
27         Mingus
1653         None
838         Louie
1276       Rodney
1667         Joey
2351         None
1279         None
87         Nugget
1626     Theodore
1455          Bob
1081      Charles
795       Bungalo
530          None
1551         Trip
2044         None
598          None
410         Wyatt
984           Blu
1060       Tucker
1849       Buddah
164          None
1611         None
266          None
2016      

In [54]:
df_archive[df_archive["dog_name"].str.islower() ==True].shape[0]

0

There are no more lower case dog names, inspection of the unique values indicates that they have been fixed.  We will trop the old `name` column and the new `dog_name` column in the next step af we copy them to our dog observation table

<div class="alert alert-block alert-info">
Tidiness - Multiple types of observational units are stored in the same table, Twitter data and Dog data are comingled
</div>


#### Define

We will merge and reconfigure the data into two files to support two observational units, one file for tweet observations and another for dog observations.  The rationale will be further elaborated on in the report document that accompanies thsi workbook.  We will take the following approach: (1) merge the df_tweet dataframe into the df_archive datafarme to get the favorites and retweet counts into this dataframe. The new df_archive_tweets dataframe will serve as the foundation of our tweet observations. (2) Merge the df_stage dataframe we melted seperately from the 4 stage columns in df_archive into the df_images datafarme to get the dog_stage into this new df_images_staging dataframe. (3) Next we'll merge these two dataframes using an INNER join and then drop all but our dog observations, and then we'll later drop the `name` column this has the effect of "moving" our `name` column to our dog observation table. This will also help account for the 12 records in the archive table that now have data in two stages. Our final stucture will provide one record per dog observation.

#### Code

In [55]:
df_archive_tweets = pd.merge(df_archive, df_tweets, how = 'left', left_on='tweet_id', right_on='id' )

In [56]:
df_images_stage = pd.merge(df_images, df_stage, how = 'left', left_on='tweet_id', right_on='tweet_id' )

In [57]:
df_dog_obs = pd.merge(df_images_stage, df_archive_tweets, how = 'inner', left_on='tweet_id', right_on='tweet_id' )

In [58]:
df_tweet_obs = df_archive_tweets.drop(['id', 'name', 'dog_name'], 1)

In [59]:
df_dog_obs = df_dog_obs.drop(['id','in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'source', 'text', 'retweeted_status_id',
       'retweeted_status_user_id', 'retweeted_status_timestamp',
       'expanded_urls', 'rating_numerator', 'rating_denominator',
       'favorite_count', 'retweet_count','name'], 1)

In [60]:
df_dog_obs=df_dog_obs.fillna({'dog_stage':'None'})

#### Test

In [61]:
df_dog_obs.columns.unique

<bound method Index.unique of Index(['tweet_id', 'jpg_url', 'img_num', 'predicted_breed',
       'confidence_interval', 'dog_stage', 'dog_name'],
      dtype='object')>

In [62]:
df_dog_obs.shape[0]

1763

In [63]:
df_tweet_obs.columns.unique

<bound method Index.unique of Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'source', 'text', 'retweeted_status_id', 'retweeted_status_user_id',
       'retweeted_status_timestamp', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'favorite_count', 'retweet_count'],
      dtype='object')>

In [64]:
df_tweet_obs.shape[0]

2356

In [65]:
df_dog_obs.head()

,tweet_id,jpg_url,img_num,predicted_breed,confidence_interval,dog_stage,dog_name
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,None,None
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,None,None
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,None,None
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,None,None
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,None,None


In [66]:
df_tweet_obs.head(5)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,favorite_count,retweet_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,39060.0,8673.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,33457.0,6372.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,25219.0,4232.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,42426.0,8776.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,40582.0,9553.0


<div class="alert alert-block alert-success">
Now we have reorganized our data based on observational units, we will use the observational dataframes to complete the remaining quality related cleanup, and then finally merge them once more into our master
</div>

<div class="alert alert-block alert-info">
Quality - Data contains retweets - Remove unnecessary rows
</div>

**Define**

Get rid of the rows that are retweets

**Code**

In [67]:
retweets = df_tweet_obs.text.str.match('^RT @')
df_tweet_obs[retweets].shape[0]

181

We have 181 retweets where the text starts with "RT @" and there are also 181 non-null rows associated with the retweeted_status_id, retweeted_status_user_id, retweeted_status_timestamp columns, before we get rid of them lets confirm that the two sets match. We'll make a list of the ids associated with each and then compare the lists

In [68]:
df_text_starts_with_RT=sorted(df_tweet_obs[retweets].retweeted_status_id.tolist())
df_retweet_status_used=sorted(df_tweet_obs[df_archive['retweeted_status_id'].notnull()]['retweeted_status_id'].tolist())
df_text_starts_with_RT==df_retweet_status_used

True

So, the sets are identical all the rows with text starting with '@RT' also have the retweet status columns populated, we'll use'RT@' to verify now that we have confirmed because its easier. 

In [69]:
# Only keep the tweets that aren't retweets
df_tweet_obs = df_tweet_obs[~retweets]

**Test**

In [70]:
retweets = df_tweet_obs.text.str.match('^RT @')
df_tweet_obs[retweets].shape[0] #should return 0

0

We should have 0 retweeted rows and we do and we should have should have **2175** remaining rows (2356 original rows minus 181 retweets).

In [71]:
df_tweet_obs.shape[0]

2175

<div class="alert alert-block alert-info">
Quality - Data contains retweets - Remove unnecessary columns
</div>

#### Define

Remove the columns that were associated with re-tweet status information because now that we have removed all of the re-tweeted rows the columns are empty and unecessary


#### Code

In [72]:
# Delete columns related to retweet we don't need anymore
df_tweet_obs = df_tweet_obs.drop('retweeted_status_id', 1)
df_tweet_obs = df_tweet_obs.drop('retweeted_status_user_id', 1)
df_tweet_obs = df_tweet_obs.drop('retweeted_status_timestamp', 1)


#### Test

In [73]:
df_tweet_obs.columns.values

array(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'source', 'text', 'expanded_urls', 'rating_numerator',
       'rating_denominator', 'favorite_count', 'retweet_count'], dtype=object)

The retweet status columns are now removed

<div class="alert alert-block alert-info">
Quality - `Source` has multiple values in one column
</div>

#### Define

Search for the primary source (there apear to be only 4) in the field and create a shorter simplified field (`tweet_source`) free of links tha can be used for analysis and visualization and remove the orgnal `source` columns

#### Code

In [74]:
# look for "Twitter Web Client", "Twitter for iPhone", "Vine - Make a Scene", or "TweetDec" in source_url:
tweet_source = []

def get_source(dataframe):
    source_url=dataframe['source']

    if "Twitter Web Client" in source_url :
        tweet_source.append("Twitter Web Client")
    elif "Twitter for iPhone" in source_url:
        tweet_source.append("Twitter for iPhone")
    elif "Vine - Make a Scene" in source_url:
        tweet_source.append("Vine - Make a Scene")
    elif "TweetDeck" in source_url:
        tweet_source.append("TweetDeck")
    else:
        tweet_source.append("None")
        
df_archive.apply(get_source, axis=1)
df_archive['tweet_source'] = tweet_source


#### Test

In [75]:
df_archive.tweet_source.unique()

array(['Twitter for iPhone', 'Twitter Web Client', 'Vine - Make a Scene',
       'TweetDeck'], dtype=object)

In [76]:
df_tweet_obs = df_tweet_obs.drop('source', 1)

In [77]:
df_tweet_obs.columns.unique

<bound method Index.unique of Index(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id', 'timestamp',
       'text', 'expanded_urls', 'rating_numerator', 'rating_denominator',
       'favorite_count', 'retweet_count'],
      dtype='object')>

<div class="alert alert-block alert-info">
Quality - `expanded_urls` has multiple values in one column and adds no value 
</div>

#### Define

Remove the `expanded_urls` column

#### Code

In [78]:
# Delete column
df_tweet_obs = df_tweet_obs.drop('expanded_urls', 1)

#### Test

In [79]:
df_tweet_obs.columns.values

array(['tweet_id', 'in_reply_to_status_id', 'in_reply_to_user_id',
       'timestamp', 'text', 'rating_numerator', 'rating_denominator',
       'favorite_count', 'retweet_count'], dtype=object)

<div class="alert alert-block alert-info">
Quality - `img_num` has no value 
</div>

#### Define

Remove the `img_num` column

#### Code

In [80]:
# Delete column
df_dog_obs = df_dog_obs.drop('img_num', 1)

#### Test

In [81]:
df_dog_obs.columns.values

array(['tweet_id', 'jpg_url', 'predicted_breed', 'confidence_interval',
       'dog_stage', 'dog_name'], dtype=object)

<div class="alert alert-block alert-info">
Quality- `timestamp` has the wrong data type
</div>


#### Define

Convert `timestamp` data type to an actual date/time type

#### Code

In [82]:
df_tweet_obs['timestamp'] = pd.to_datetime(df_tweet_obs.timestamp)

#### Test

In [83]:
print(df_tweet_obs['timestamp'].dtype)

datetime64[ns]


<div class="alert alert-block alert-info">
Qality - Data types for in_reply_to_status_id and in_reply_to_user_id could be changed to match Twitter API  
Quality - `in_reply_to_status_id` and `in_reply_to_user_id` are sparsely populated
</div>

#### Define

My original thoughts regarding these fields were to convert the data types to match the Twitter API reference. However, these fields had only 78 records to begin with and only 21 after cleaning, we will drop these fields

#### Code

In [84]:
# Delete column
df_tweet_obs = df_tweet_obs.drop(['in_reply_to_status_id','in_reply_to_user_id'], 1)

#### Test

<div class="alert alert-block alert-info">
Quality - Dog breed names are seperated by underscore and have inconsistent case
</div>

#### Define

Replace the underscore with a space and adjust case

#### Code

In [85]:
df_dog_obs['predicted_breed'] = df_dog_obs['predicted_breed'].str.strip().str.replace('_', ' ')
df_dog_obs['predicted_breed'] = df_dog_obs['predicted_breed'].str.title()


#### Test

In [86]:
# NOTE: This call was originally a call to .unique, however the output 
# was too long and was apparently corrupting my notebook HTML save output
# please note that more data was inspected and this chnage is a workaround
# to save the file as HTML
df_dog_obs['predicted_breed'].sample(75)

395               Golden Retriever
930                            Pug
1663            Labrador Retriever
1645                        Basset
270       Chesapeake Bay Retriever
237                Irish Wolfhound
1084                      Airedale
1512            Labrador Retriever
1338              Golden Retriever
1579            Labrador Retriever
1393     Staffordshire Bullterrier
507             Labrador Retriever
654                            Pug
362                         Borzoi
1661                    Eskimo Dog
1558                        Kelpie
913                     Toy Poodle
1110              Golden Retriever
1617                      Pembroke
1207                       Clumber
485                 French Bulldog
1184          Old English Sheepdog
1127                          Chow
943                 French Bulldog
993                       Malamute
1428                        Kelpie
730              Yorkshire Terrier
117                       Komondor
548                 

## Store Data

In [87]:
df_master = pd.merge(df_tweet_obs, df_dog_obs, how = 'inner', left_on='tweet_id', right_on='tweet_id' )

In [88]:
df_master.columns.unique

<bound method Index.unique of Index(['tweet_id', 'timestamp', 'text', 'rating_numerator',
       'rating_denominator', 'favorite_count', 'retweet_count', 'jpg_url',
       'predicted_breed', 'confidence_interval', 'dog_stage', 'dog_name'],
      dtype='object')>

In [89]:
df_master.shape[0]

1696

In [90]:
# Store the clean DataFrame in a CSV file
df_master.to_csv('twitter_archive_master.csv', index=False, encoding = 'utf-8')
df_tweet_obs.to_csv('tweet_observaton.csv', index=False, encoding = 'utf-8')
df_dog_obs.to_csv('dog_observation.csv', index=False, encoding = 'utf-8')

## Insights and Visualizations

<div class="alert alert-block alert-success">
Insights and visualizations are included in the accompanying act_report.ipnb notebook
</div>


## References

Capitalize first letter of each word in the column Python
http://bit.ly/2DhYMrw

pandas.DataFrame.fillna 
http://bit.ly/2p2j83C

Merge, Join, Concatenate
http://bit.ly/2u8jpDM

Pandas Melt several groups of columns into multiple target columns by name
http://bit.ly/2GevM6Z